---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [50]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [51]:
def date_sorter():
    
    df=df.apply(lambda x:x.strip())
    
    # try to grab all numbers.  however, random number 5060,64, or Mar 2009 can still be extract from this
    df1=df.str.extractall(r'((?:\d{1,2}[/-])?(?:\d{1,2}[/-])?(?:\d{2,4}))').reset_index()  
    df1.columns=['index', 'match', 'Date']
    df1=df1[df1['Date'].apply(lambda x:len(x)>=4)]  # filter out len of number <=2, and match!=0
    df1=df1.loc[df1.groupby('index')['match'].idxmin()]
    df1.set_index(keys='index',inplace=True)
    df1.drop(labels='match',axis=1,inplace=True)
    df1['Date_1']=df1['Date'].apply(lambda x:pd.to_datetime(x,errors='coerce'))
    df1.loc[80,'Date_1']=pd.to_datetime('6/29/81')
    df1.loc[99,'Date_1']=pd.to_datetime('11/14/83')
    df1.loc[248,'Date_1']=pd.to_datetime('1 July 1995')
    df1.loc[271,'Date_1']=pd.to_datetime('1 August 2008')
    df1.loc[272,'Date_1']=pd.to_datetime('1 Feb 1993')
    
    df2=df.str.extractall(r'((?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\,| )?(?:\d{1,2}[a-z]{,2})?(?:-|\,| )?\d{4})')
    df2.reset_index(inplace=True)
    df2.columns=['index', 'match', 'Date']
    df2.set_index(keys='index',inplace=True)
    df2.drop(labels='match',axis=1,inplace=True)
    df2['Date_2']=df2['Date'].apply(lambda x:pd.to_datetime(x,errors='coerce'))
    df2.loc[298,'Date_2']=pd.to_datetime('1 Jan 1993')
    df2.loc[313,'Date_2']=pd.to_datetime('1 Dec 1978')
    
    df_all=df1.merge(df2,how='outer',left_index=True,right_index=True)
    df_all['D2']=df_all['Date_2'].isnull()
    df_all=df_all.apply(lambda row: row['Date_1'] if row['D2']==True else row['Date_2'], axis=1)
    
    index_list=list(df_all.sort_values().index)
    index_value=list(range(500))
    result={'index':index_list,'order':index_value}
    result=pd.DataFrame(data=result)

    
    return (result.sort_values(by='index').set_index('index'))

In [52]:
df.head(20)

0          03/25/93 Total time of visit (in minutes):\n
1                        6/18/85 Primary Care Doctor:\n
2     sshe plans to move as of 7/8/71 In-Home Servic...
3                 7 on 9/27/75 Audit C Score Current:\n
4     2/6/96 sleep studyPain Treatment Pain Level (N...
5                     .Per 7/06/79 Movement D/O note:\n
6     4, 5/18/78 Patient's thoughts about current su...
7     10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                          3/7/86 SOS-10 Total Score:\n
9              (4/10/71)Score-1Audit C Score Current:\n
10    (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                        4/09/75 SOS-10 Total Score:\n
12    8/01/98 Communication with referring physician...
13    1/26/72 Communication with referring physician...
14    5/24/1990 CPT Code: 90792: With medical servic...
15    1/25/2011 CPT Code: 90792: With medical servic...
16          4/12/82 Total time of visit (in minutes):\n
17         1; 10/13/1976 Audit C Score, Highest/

In [53]:
df=df.apply(lambda x:x.strip())

In [54]:
type(df)

pandas.core.series.Series

In [55]:
# try to grab all numbers.  however, random number 5060,64, or Mar 2009 can still be extract from this
df1=df.str.extractall(r'((?:\d{1,2}[/-])?(?:\d{1,2}[/-])?(?:\d{2,4}))').reset_index()  

In [56]:
df1.columns=['index', 'match', 'Date']

In [57]:
## keep the longest len of Date, so we create a new column that has the length of date
df1['dlen']=df1['Date'].apply(lambda x:len(x))

In [58]:
## keep the longest len of Date
df1=df1.loc[df1.groupby('index')['dlen'].idxmax()]

In [59]:
df1.set_index(keys='index',inplace=True)

In [60]:
df1.drop(labels='match',axis=1,inplace=True)

In [61]:
df1.loc[6]

Date    5/18/78
dlen          7
Name: 6, dtype: object

In [21]:
#df1['Date'].str.extractall(r'(\d{1,2}/\d{1,2}/\d{2})')

In [62]:
df1['Date_1']=df1['Date'].apply(lambda x:pd.to_datetime(x,errors='coerce'))

In [63]:
df1[df1['Date_1'].isnull()]  # only need to update index 72 and 99

,Date,dlen,Date_1
index,,,
72,9/36/308,8,NaT
99,14/83,5,NaT
248,50-100,6,NaT
271,7787,4,NaT
272,11-16,5,NaT


In [100]:
df2=df.str.extractall(r'((?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\,| |\. )?(?:\d{1,2}[a-z]{,2})?(?:-|\,| |\, )?\d{4})')

In [101]:
df2.reset_index(inplace=True)
df2.columns=['index', 'match', 'Date']
df2.set_index(keys='index',inplace=True)
df2.drop(labels='match',axis=1,inplace=True)
df2.head()

,Date
index,
125,24 Jan 2001
126,10 Sep 2004
127,26 May 1982
128,28 June 2002
129,06 May 1972


In [102]:
df2.shape

(218, 1)

In [67]:
df2['Date_2']=df2['Date'].apply(lambda x:pd.to_datetime(x,errors='coerce'))

In [68]:
df2[df2['Date_2'].isnull()]  # mis-spell

,Date,Date_2
index,,
298,Janaury 1993,NaT
313,Decemeber 1978,NaT


In [69]:
df2.loc[298,'Date_2']=pd.to_datetime('1 Jan 1993')
df2.loc[313,'Date_2']=pd.to_datetime('1 Dec 1978')

In [70]:
df_all=df1.merge(df2,how='outer',left_index=True,right_index=True)

In [71]:
df_all.head(10)

,Date_x,dlen,Date_1,Date_y,Date_2
index,,,,,
0,03/25/93,8,1993-03-25,NaN,NaT
1,6/18/85,7,1985-06-18,NaN,NaT
2,7/8/71,6,1971-07-08,NaN,NaT
3,9/27/75,7,1975-09-27,NaN,NaT
4,2/6/96,6,1996-02-06,NaN,NaT
5,7/06/79,7,1979-07-06,NaN,NaT
6,5/18/78,7,1978-05-18,NaN,NaT
7,10/24/89,8,1989-10-24,NaN,NaT
8,3/7/86,6,1986-03-07,NaN,NaT


In [72]:
df_all['D2']=df_all['Date_2'].isnull()

In [73]:
df_all.head()

,Date_x,dlen,Date_1,Date_y,Date_2,D2
index,,,,,,
0,03/25/93,8,1993-03-25,NaN,NaT,True
1,6/18/85,7,1985-06-18,NaN,NaT,True
2,7/8/71,6,1971-07-08,NaN,NaT,True
3,9/27/75,7,1975-09-27,NaN,NaT,True
4,2/6/96,6,1996-02-06,NaN,NaT,True


In [75]:
df_all['F_Date']=df_all.apply(lambda row: row['Date_1'] if row['D2'] else row['Date_2'],axis=1)

In [76]:
df_all.loc[226]

Date_x                   1991
dlen                        4
Date_1    1991-01-01 00:00:00
Date_y       January 07, 1991
Date_2    1991-01-07 00:00:00
D2                      False
F_Date    1991-01-07 00:00:00
Name: 226, dtype: object

In [77]:
index_list=df_all.sort_values(by='F_Date').index

In [83]:
df_all['F_Date'][0].month

3

In [85]:
sum(df_all['F_Date'].apply(lambda x: (x.month==1) and (x.day==1)))

69

In [86]:
sum(df_all['F_Date'].apply(lambda x: (x.day==1)))

280

In [104]:
df.loc[358]

'.Mother died, 1/1983.'

In [88]:
df_all[df_all['F_Date'].apply(lambda x: (x.month==1) and (x.day==1))]

,Date_x,dlen,Date_1,Date_y,Date_2,D2,F_Date
index,,,,,,,
198,2013,4,2013-01-01,NaN,NaT,True,2013-01-01
204,1976,4,1976-01-01,NaN,NaT,True,1976-01-01
214,1974,4,1974-01-01,NaN,NaT,True,1974-01-01
227,2011,4,2011-01-01,NaN,NaT,True,2011-01-01
238,1995,4,1995-01-01,January 1995,1995-01-01,False,1995-01-01
244,2013,4,2013-01-01,"January, 2013",2013-01-01,False,2013-01-01
252,2007,4,2007-01-01,Jan 2007,2007-01-01,False,2007-01-01
284,1987,4,1987-01-01,Jan 1987,1987-01-01,False,1987-01-01
286,2013,4,2013-01-01,January 2013,2013-01-01,False,2013-01-01


In [74]:
df.loc[226]

'See previous - Parking garage incident January 07, 1991.'

In [78]:
pd.Series(index_list)

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     415
17     335
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     214
28     436
29     104
      ... 
470    243
471    139
472    320
473    383
474    244
475    198
476    286
477    480
478    431
479    279
480    381
481    463
482    366
483    439
484    255
485    401
486    475
487    257
488    152
489    235
490    464
491    253
492    231
493    427
494    141
495    186
496    161
497    413
498     72
499     99
Name: index, Length: 500, dtype: int64

In [72]:
index_value=list(range(500))

In [73]:
result={'index':index_list,'order':index_value}
result=pd.DataFrame(data=result)
result.sort_values(by='index').set_index('index')

,order
index,
0,270
1,168
2,2
3,55
4,310
5,98
6,88
7,226
8,178


,order
index,
0,272
1,170
2,2
3,55
4,312
5,99
6,89
7,228
8,180
